In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from torch.utils.data import DataLoader, Dataset
import json
import wandb

/root/.local/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2022-05-10 19:38:39.601349: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer.so.6'; dlerror: libnvinfer.so.6: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2022-05-10 19:38:39.601494: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer_plugin.so.6'; dlerror: libnvinfer_plugin.so.6: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2022-05-10 19:38:39.601506: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:30] Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU w

In [2]:
!nvidia-smi

Tue May 10 19:38:54 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.51.05    Driver Version: 450.51.05    CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-PCIE...  On   | 00000000:41:00.0 Off |                    0 |
| N/A   28C    P0    23W / 250W |      0MiB / 32510MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
wandb.init(project="seq2seq", entity='cocoshe')


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: cocoshe (use `wandb login --relogin` to force relogin)
2022-05-10 19:38:59.861883: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer.so.6'; dlerror: libnvinfer.so.6: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2022-05-10 19:38:59.862018: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer_plugin.so.6'; dlerror: libnvinfer_plugin.so.6: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2022-05-10 19:38:59.862032: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:30] Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, pleas

In [4]:

json_data = []
with open('dataset/train.json', 'r',encoding='utf-8') as f:
    for line in f:
        data = json.loads(line)
        json_data.append(data)
json_data[:10]

[{'content': '类型#裤*版型#宽松*风格#性感*图案#线条*裤型#阔腿裤',
  'summary': '宽松的阔腿裤这两年真的吸粉不少，明星时尚达人的心头爱。毕竟好穿时尚，谁都能穿出腿长2米的效果宽松的裤腿，当然是遮肉小能手啊。上身随性自然不拘束，面料亲肤舒适贴身体验感棒棒哒。系带部分增加设计看点，还让单品的设计感更强。腿部线条若隐若现的，性感撩人。颜色敲温柔的，与裤子本身所呈现的风格有点反差萌。'},
 {'content': '类型#裙*风格#简约*图案#条纹*图案#线条*图案#撞色*裙型#鱼尾裙*裙袖长#无袖',
  'summary': '圆形领口修饰脖颈线条，适合各种脸型，耐看有气质。无袖设计，尤显清凉，简约横条纹装饰，使得整身人鱼造型更为生动立体。加之撞色的鱼尾下摆，深邃富有诗意。收腰包臀,修饰女性身体曲线，结合别出心裁的鱼尾裙摆设计，勾勒出自然流畅的身体轮廓，展现了婀娜多姿的迷人姿态。'},
 {'content': '类型#上衣*版型#宽松*颜色#粉红色*图案#字母*图案#文字*图案#线条*衣样式#卫衣*衣款式#不规则',
  'summary': '宽松的卫衣版型包裹着整个身材，宽大的衣身与身材形成鲜明的对比描绘出纤瘦的身形。下摆与袖口的不规则剪裁设计，彰显出时尚前卫的形态。被剪裁过的样式呈现出布条状自然地垂坠下来，别具有一番设计感。线条分明的字母样式有着花式的外观，棱角分明加上具有少女元气的枣红色十分有年轻活力感。粉红色的衣身把肌肤衬托得很白嫩又健康。'},
 {'content': '类型#裙*版型#宽松*材质#雪纺*风格#清新*裙型#a字*裙长#连衣裙',
  'summary': '踩着轻盈的步伐享受在午后的和煦风中，让放松与惬意感为你免去一身的压力与束缚，仿佛要将灵魂也寄托在随风摇曳的雪纺连衣裙上，吐露出<UNK>微妙而又浪漫的清新之意。宽松的a字版型除了能够带来足够的空间，也能以上窄下宽的方式强化立体层次，携带出自然优雅的曼妙体验。'},
 {'content': '类型#上衣*材质#棉*颜色#蓝色*风格#潮*衣样式#polo*衣领型#polo领*衣袖长#短袖*衣款式#拼接',
  'summary': '想要在人群中脱颖而出吗？那么最适合您的莫过于这款polo衫短袖，采用了经典的polo领口和柔软纯棉面料，让您紧跟时尚

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [6]:
class MyDataset(Dataset):
    def __init__(self, input_data, tokenizer):
        super(MyDataset, self).__init__()
        self.input_data = input_data
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.input_data)

    def __getitem__(self, idx):
        item = self.input_data[idx]
        content, summary = item['content'], item['summary']
        # con_sum = [content, summary]
        tokens_content = self.tokenizer(content, padding='max_length', max_length=128, truncation=True, return_tensors='pt', return_token_type_ids=True)
        tokens_summary = self.tokenizer(summary, padding='max_length', max_length=128, truncation=True, return_tensors='pt', return_token_type_ids=True)
        input_ids = tokens_content['input_ids'].squeeze(0)
        input_ids_summary = tokens_summary['input_ids'].squeeze(0)
        attention_mask = tokens_content['attention_mask'].squeeze(0)
        # print(input_ids.shape, token_type_ids.shape, attention_mask.shape)
        return input_ids, attention_mask, input_ids_summary

In [7]:
tokenizer = AutoTokenizer.from_pretrained("bart-base-chinese-cluecorpussmall")
model = AutoModelForSeq2SeqLM.from_pretrained("bart-base-chinese-cluecorpussmall").to(device)


In [8]:

token_test = tokenizer(["我是中国人", "中国人是我"], padding='max_length', max_length=128, truncation=True, return_tensors='pt', return_token_type_ids=True)
token_test.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [9]:
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(json_data, test_size=0.2, random_state=42, shuffle=True)
train_dataloader = DataLoader(MyDataset(train_data, tokenizer), batch_size=64, shuffle=True)
test_dataloader = DataLoader(MyDataset(test_data, tokenizer), batch_size=64, shuffle=True)

In [10]:
epochs = 5
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.9)

In [11]:
def train():
    model.train()
    for epoch in range(epochs):
        for idx, batch in enumerate(tqdm(train_dataloader)):
            input_ids, attention_mask, input_ids_summary = batch
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            input_ids_summary = input_ids_summary.to(device)
            outputs = model(input_ids, attention_mask=attention_mask, labels=input_ids_summary)
            loss = outputs[0]

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            wandb.log({'epoch': epoch, 'batch': idx, 'loss': loss.item()})

            if idx % 100 == 0:
                print('epoch: {}, loss: {}'.format(epoch, loss.item()))
        scheduler.step()

In [12]:
def evaluate():
    model.eval()
    with torch.no_grad():
        for idx, batch in enumerate(tqdm(test_dataloader)):
            input_ids, attention_mask, input_ids_summary = batch
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            input_ids_summary = input_ids_summary.to(device)
            outputs = model(input_ids, attention_mask=attention_mask, labels=input_ids_summary)
            loss = outputs[0]
            print('loss: {}'.format(loss.item()))

In [13]:
train()
evaluate()

  0%|          | 1/1433 [00:01<27:10,  1.14s/it]

epoch: 0, loss: 8.511828422546387


  7%|▋         | 101/1433 [01:07<15:03,  1.47it/s]

epoch: 0, loss: 2.3995165824890137


 14%|█▍        | 201/1433 [02:13<13:17,  1.55it/s]

...

loss: 1.7683753967285156


 99%|█████████▉| 357/359 [01:45<00:00,  3.30it/s]

loss: 1.694414496421814


100%|██████████| 359/359 [01:46<00:00,  3.38it/s]

loss: 1.7103379964828491
loss: 1.7174614667892456


In [14]:
torch.save(model.state_dict(), './model/bart_gen_ads.pth')